In [1]:
import openai
from dotenv import load_dotenv
import os
import pprint
import json
from pydantic import BaseModel, Field
from typing import Optional, List

load_dotenv("../../.env")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
# define function structure in pydantic

class ExtractJoblistParams(BaseModel):
    job_suggestions: List[str] = Field(..., description="Job title plus short description")

In [3]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo-0613", 
                                 temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        functions=[
        {
          "name": "job_list",
          "description": "Generate a list with 5 job recommendations",
          "parameters": ExtractJoblistParams.schema()
        }
    ],
        function_call="auto",
    )
    print(response)
    function_call = response.choices[0].message["function_call"]
    arguments = json.loads(function_call["arguments"])
    return arguments

In [6]:
delimiter = "####"
system_message = f"""
You are a job coach. Give me list of 5 job recommendations

"""

messages =  [  
{'role':'system', 'content': f"{system_message}"},    
] 
response = get_completion_from_messages(messages)
pprint.pprint(response)

{
  "id": "chatcmpl-7XnG9X8M1esbDeLTuIxysuQ8IgaCO",
  "object": "chat.completion",
  "created": 1688288533,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "job_list",
          "arguments": "{\n  \"job_suggestions\": [\n    \"Software Engineer: Develop and maintain software applications\",\n    \"Marketing Manager: Plan and execute marketing campaigns\",\n    \"Graphic Designer: Create visual designs for various mediums\",\n    \"Data Analyst: Analyze and interpret complex data sets\",\n    \"Sales Representative: Build and maintain client relationships\"\n  ]\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 69,
    "completion_tokens": 75,
    "total_tokens": 144
  }
}
{'job_suggestions': ['Software Engineer: Develop and maintain software '
                     'applications',
          